# Generating synthetic data for projects

## Preparing data

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from numpy import datetime64
import warnings
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sdv.tabular import GaussianCopula, CTGAN, CopulaGAN, TVAE

In [2]:
df=pd.read_excel('Project_date_v3.xlsx', header = 1)

In [3]:
df

,№,GROUP,INITIATIVE CODE,CATEGORY,BUDGET >5 MLN,BUDGET MLN,DATE START PLAN,DATE END PLAN,DURATION PLAN,DURATION >30,COMPLIANCE STRATEGY,AUTHORIZATION,FINISH FACT,REQUIREMENTS LAW,PM NAME,SPONSOR NAME
0,1,UB,UB_0_008,STRAT,1,7.18,2016-06-17,2018-12-28,30,1,1,1,1,1,William T.,Bill E.
1,2,SР,SР_1_004,USUAL,0,2.55,2015-07-01,2016-08-30,13,0,0,1,1,0,Thomas M.,Jerry H.
2,3,OB,OB_1_009,USUAL,0,3.68,2015-07-28,2019-04-30,45,1,1,1,1,1,Scarlett Y.,Gordon D.
3,4,UB,UB_0_004,STRAT,1,5.53,2015-12-24,2016-12-30,12,0,1,0,0,0,Samuel.J.,Bill E.
4,5,РP,РP_1_044,USUAL,0,4.34,2020-03-19,2021-10-31,19,0,1,1,1,1,Phoebe F.,Dave P.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,151,UP,UP_1_006,USUAL,0,2.03,2015-07-01,2016-12-30,17,0,1,1,1,1,Jacob G.,Brad P.
151,152,UP,UP_1_010,USUAL,0,1.78,2016-04-01,2016-12-30,8,0,1,1,1,0,Maryam E.,Brad P.
152,153,OB,ОB_0_012,STRAT,1,7.54,2015-08-20,2018-12-31,40,1,1,1,1,0,Bethany S.,Michael J.
153,154,OB,ОB_1_013,USUAL,0,4.95,2015-09-01,2016-06-30,9,0,1,1,1,1,Chloe L.,Michael J.


In [4]:
# Add column with project duration in days
df['DAYS'] = df['DATE END PLAN'] - df['DATE START PLAN']
# Convert timedelta to int (days)
df['DAYS'] = df['DAYS'] // np.timedelta64(1, 'D')

In [5]:
# Set duration plan as difference between start date abd end date (month)
# We need to update original column 'DURATION PLAN' because it was calculated in excel by DATEDIF formula.
# In some rare cases this formula returns different results from calculated in Python. 
# But for data generation original and calculated values have to be equal.

def duration_plan(df) -> int:
    return (df['DATE END PLAN'] - df['DATE START PLAN']) // np.timedelta64(1, 'M')

In [6]:
df['DURATION PLAN'] = duration_plan(df)

In [7]:
# Let's have a look on data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155 entries, 0 to 154
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   №                    155 non-null    int64         
 1   GROUP                155 non-null    object        
 2   INITIATIVE CODE      155 non-null    object        
 3   CATEGORY             155 non-null    object        
 4   BUDGET >5 MLN        155 non-null    int64         
 5   BUDGET MLN           155 non-null    float64       
 6   DATE START PLAN      155 non-null    datetime64[ns]
 7   DATE END PLAN        155 non-null    datetime64[ns]
 8   DURATION PLAN        155 non-null    int64         
 9   DURATION >30         155 non-null    int64         
 10  COMPLIANCE STRATEGY  155 non-null    int64         
 11  AUTHORIZATION        155 non-null    int64         
 12  FINISH FACT          155 non-null    int64         
 13  REQUIREMENTS LAW     155 non-null  

In [8]:
# Drop unique columns, which we will not use in machine learning
df = df.drop(['№', 'INITIATIVE CODE'], axis = 1)

In [9]:
# Split data into train for generating and hold-out set
df_gen, df_test = train_test_split(df, test_size=0.2, random_state=8)

In [10]:
# Save hold-out set
df_test.to_csv("test_data.csv")

## Set Constraints

In [11]:
# imports
from sdv.constraints import ColumnFormula

In [12]:
# Set DATE END PLAN as DATE START PLAN plus DAYS
def date_end_plan(df) -> datetime64:
    return df['DATE START PLAN'] + pd.to_timedelta(df['DAYS'], unit='D')

date_end_plan_constraint = ColumnFormula(
    column='DATE END PLAN',
    formula=date_end_plan,
    handling_strategy='transform'
)

In [13]:
# Set duration plan as difference between start date abd end date (month)

duration_plan_constraint = ColumnFormula(
    column='DURATION PLAN',
    formula=duration_plan,
    handling_strategy='transform'    
)

In [14]:
# set Budget > 5 MLN feature
def budget_gt_5mln(df) -> int:
    return df['BUDGET MLN'].apply(lambda x: 1 if x > 5 else 0)

budget_gt_5mln_constraint = ColumnFormula(
    column='BUDGET >5 MLN',
    formula=budget_gt_5mln,
    handling_strategy='transform'
)

In [15]:
# set duration more or equal 30 month constraint
def duration_ge_30(df) -> int:
    return df['DURATION PLAN'].apply(lambda x: 1 if x >= 30 else 0)

duration_ge_30_constraint = ColumnFormula(
    column='DURATION >30',
    formula=duration_ge_30,
    handling_strategy='transform'
)

In [16]:
constraints = [budget_gt_5mln_constraint, duration_ge_30_constraint, duration_plan_constraint, date_end_plan_constraint]

## Set other parametres

In [17]:
#Set transformer for categorical columns with relatively big number of groups. 
#For columns 'GROUP' and 'CATEGORY' by default transformer is One Hot Encoder

field_transformers = {
    'PM NAME': 'label_encoding',
    'SPONSOR NAME': 'label_encoding'    
}

## Generate synthetic data

In [18]:
# Generate synthetic data
def generate_data(real_data: pd.DataFrame) -> dict:
    args = dict(rounding=2, field_transformers=field_transformers, constraints=constraints)
    models = [
                ('GaussianCopula', GaussianCopula(**args)),
                ('CTGAN', CTGAN(**args)),
                ('CopulaGAN', CopulaGAN(**args)),
                ('TVAE', TVAE(**args))
             ]
    
    res = dict()
    for name, model in models:
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            model.fit(real_data)
        synth_data = model.sample(2000)
        res[name] = synth_data
    
    return res

In [19]:
# Get synthetic data
synth_data_dict = generate_data(df_gen)

## Synthetic data evaluation

https://sdv.dev/SDV/user_guides/evaluation/single_table_metrics.html

Statistical metrics

In [20]:
# For continuos columns will use two-sample Kolmogorov–Smirnov test.
# For discrete columns will use Chi-squared test.
# In each case, the statistical test will be executed on all the compatible column 
# (so, categorical or boolean columns for CSTest and numerical columns for KSTest), 
# and report the average score obtained.
from sdv.metrics.tabular import CSTest, KSTest

Likelihood metrics

In [21]:
from sdv.metrics.tabular import BNLikelihood

Detection Metrics

In [22]:
# We will use logistic regression and SVC classifier
from sdv.metrics.tabular import LogisticDetection, SVCDetection

In [23]:
#distributions = model.get_distributions()
#distributions

In [24]:
# Get % of target variable
def get_target(synth_data: pd.DataFrame, target: str) -> tuple:
    percent = synth_data[target].value_counts(normalize = True)
    return (percent[0], percent[1])

In [25]:
# setup metrics
def get_metrics(real_data: pd.DataFrame, synth_data: pd.DataFrame) -> pd.DataFrame:
    metrics = [
                ('target', get_target(synth_data, 'AUTHORIZATION')),
                ('CSTest', CSTest.compute(real_data, synth_data)),
                ('KSTest', KSTest.compute(real_data, synth_data)),
                ('BNLikelihood', BNLikelihood.compute(real_data, synth_data)),
                ('LogisticDetection', LogisticDetection.compute(real_data, synth_data)),
                ('SVCDetection', SVCDetection.compute(real_data, synth_data))
    ]
    
    result = []
    names = []
    for name, metric in metrics:
        names.append(name)
        result.append(metric)
        
    return pd.DataFrame([result], columns=names)

In [26]:
# Evaluate synthetic data, obtained from different models, by different metrics
def exp_run(real_data: pd.DataFrame, synth_data: dict) -> pd.DataFrame:
    names = []
    results = []
    for name, sdata in synth_data.items():
        results.append(get_metrics(real_data, sdata))
        names.append(name)
                
    return pd.concat(results).set_axis(names)

In [27]:
exp_run(df_gen, synth_data_dict)

,target,CSTest,KSTest,BNLikelihood,LogisticDetection,SVCDetection
GaussianCopula,"(0.454, 0.546)",0.882383,0.822781,0.000276,0.054615,0.073394
CTGAN,"(0.261, 0.739)",0.944192,0.799020,0.000785,0.269758,0.094483
CopulaGAN,"(0.058, 0.942)",0.950459,0.846552,0.001299,0.262415,0.131475
TVAE,"(0.027, 0.973)",0.940919,0.837835,0.003674,0.341948,0.221455


In [28]:
# The best result give GaussianCopula
# Save all synthetic data which we will use in further experiments
for name, sdata in synth_data_dict.items():
    sdata.to_csv(f'df_synth_{name}.csv')